В предишната терадка установхиме, че обработката на цели статии не използва ефективно GPU-то. Освен това скоростта на учене не беше добра - вероятно заради малкия размер на batch-овете.

Ще опитаме да използваме batch-ове с по-голям размер. За целта ще съставим batch-овете от малки парчета от статиит. Ще използваме stateful LSTM модел, за да запазим състоянието на LSTM клетките между batch-ове съставящи една и съща статия.

След последния batch от дадена статия ще рестартираме състоянието на клетките.

In [1]:
import os

import tensorflow as tf
import numpy as np

In [38]:
BATCH_SIZE = 64
BATCHED_ITEM_LENGTH = 128
BUFFER_SIZE = 256

with open('page_revisions_text', 'rb') as text_file:
    data = text_file.read()

articles = sorted(data.split(b'\0')[:2000], key=len)

def articles_generator():
    for index, article in enumerate(articles):
        yield np.frombuffer(article + b'\0', dtype=np.uint8)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.frombuffer(b'\0', dtype=np.uint8)
        index += 1
        
def subbatches():
    dataset = tf.data.Dataset.from_generator(articles_generator, output_types=tf.uint8)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None]), drop_remainder=True)

    for batch in dataset.as_numpy_iterator():
        remaining = batch
        while remaining.shape[1] > 1:
            yield remaining[:, :BATCHED_ITEM_LENGTH]
            remaining = remaining[:, BATCHED_ITEM_LENGTH-1:]

dataset = tf.data.Dataset.from_generator(subbatches, output_types=tf.uint8, output_shapes=(BATCH_SIZE, None))
dataset = dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

dataset

<MapDataset shapes: ((64, None), (64, None)), types: (tf.uint8, tf.uint8)>

In [39]:
vocab_size = 256
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [40]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [41]:
from tensorflow.keras import metrics

def average_batch_length(true_labels, predictions):
    return tf.shape(true_labels)[1]

model = build_model(vocab_size = vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

In [42]:
checkpoint_dir = './training_checkpoints' # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [81]:
class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        self.last_total_length = 0

    def on_batch_end(self, batch, logs={}):
        total_length = int(round(logs['average_batch_length'] * (batch + 1)))
        current_batch_length = total_length - self.last_total_length
        self.last_total_length = total_length
        
        if current_batch_length < BATCHED_ITEM_LENGTH - 1:
            self.model.reset_states()
        
model_state_resetter_callback = ModelStateResetter()

In [82]:
total_epochs = 12

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.reset_states()
    history = model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/12
4938/4938 [==============================] - 308s 62ms/step - loss: 0.4330 - average_batch_length: 126.5282
Epoch 2/12
4730/4730 [==============================] - 295s 62ms/step - loss: 0.4335 - average_batch_length: 126.5421
Epoch 3/12
4975/4975 [==============================] - 311s 62ms/step - loss: 0.3984 - average_batch_length: 126.5495
Epoch 4/12
4858/4858 [==============================] - 300s 62ms/step - loss: 0.3988 - average_batch_length: 126.5216
Epoch 5/12
4592/4592 [==============================] - 285s 62ms/step - loss: 0.4131 - average_batch_length: 126.5094
Epoch 6/12
4720/4720 [==============================] - 295s 63ms/step - loss: 0.3964 - average_batch_length: 126.5038
Epoch 7/12
4554/4554 [==============================] - 285s 63ms/step - loss: 0.4051 - average_batch_length: 126.5553
Epoch 8/12
4747/4747 [==============================] - 297s 63ms/step - loss: 0.3851 - average_batch_length: 126.4609
Epoch 9/12
4785/4785 [==========================

In [83]:
with open('page_revisions_text', 'rb') as text_file:
    data = text_file.read()

articles = data.split(b'\0')[:2000]
del data

article = articles[120]
del articles

len(article)

25541

In [84]:
import huffman

def huffman_archive_size(model, text):
    archived_size = 0
    ones = 0
    input_eval = [s for s in b' ']
    input_eval = tf.expand_dims(input_eval, 0)
  
    # Empty string to store our results
    text_generated = []

    model.reset_states()

    for byte in text:
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
  
        probabilities = tf.nn.softmax(predictions[0])
        codebook = huffman.codebook([index, tensor.numpy()] for index, tensor in enumerate(probabilities))

        code = codebook[byte]
        ones += code.count('1')
        archived_size += len(code)

        input_eval = tf.expand_dims([byte], 0)
  
    return ones, archived_size

In [85]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [86]:
ones, archived_size = huffman_archive_size(model, article)
print('\nTotal length:', archived_size)

compression_ratio = archived_size / (len(article) * 8)
print('Compression ratio:', compression_ratio)

k = (ones / archived_size)
arithmetic_compression_ratio = compression_ratio * (-k * np.log2(k) - (1-k) * np.log2(1-k))
print('Potential compression ratio with arithmetic coding:', arithmetic_compression_ratio)


Total length: 50077
Compression ratio: 0.24508143768842253
Potential compression ratio with arithmetic coding: 0.22664568670800883
